In [1]:
!pip3 install deeppavlov
!pip install --upgrade numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.18.0-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.21.2 which is incompatible.
xarray 0.20.2 requires pandas>=1.1, but you have pandas 0.25.3 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.18.0 which is incompatible.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.
tensorflow 2.8.2+zzzcolab20220527125636 requires numpy>=1.20, but you have numpy 1.18.0 which is incompatible.
tab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.0
    Uninstalling numpy-1.18.0:
      Successfully uninstalled numpy-1.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.21.2 which is incompatible.
xarray 0.20.2 requires pandas>=1.1, but you have pandas 0.25.3 which is incompatible.
thinc 8.0.17 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.
spacy 3.3.1 requires pydantic!=1.8,!=1.8.1,<1.9.0,>=1.7.4, but you have pydantic 1.3 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-le

In [2]:
import pandas as pd
import numpy as np
import ast
import csv

from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator
from deeppavlov.models.preprocessors.str_lower import str_lower
from deeppavlov.models.tokenizers.nltk_moses_tokenizer import NLTKMosesTokenizer
from deeppavlov.core.data.utils import simple_download


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdo

In [ ]:
np.random.seed(365)

In [3]:
df = pd.read_csv("./data/train.csv")

In [4]:
simple_download(url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt", 
                destination="./glove.6B.100d.txt")

2022-06-21 16:25:40.447 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to glove.6B.100d.txt
347MB [00:07, 43.5MB/s]


In [5]:
from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

In [6]:
embedder = GloVeEmbedder(load_path='./glove.6B.100d.txt',
                         dim=100, pad_zero=True)

2022-06-21 16:25:49.718 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/glove.6B.100d.txt`]


### Model

In [7]:
from deeppavlov.metrics.accuracy import sets_accuracy

In [11]:
cnn_config = {
  "dataset_reader": {
    "class_name": "basic_classification_reader",
    "x": "title",
    "y": "keywords",
    "data_path": "./data",
    "class_sep": ", "
  },
  "dataset_iterator": {
    "class_name": "basic_classification_iterator",
    "seed": 42,
    "split_seed": 23,
    "field_to_split": "train",
    "split_fields": [
      "train",
      "valid"
    ],
    "split_proportions": [
      0.9,
      0.1
    ]
  },
  "chainer": {
    "in": [
      "x"
    ],
    "in_y": [
      "y"
    ],
    "pipe": [
      {
        "id": "classes_vocab",
        "class_name": "simple_vocab",
        "fit_on": [
          "y"
        ],
        "level": "token",
        "save_path": "./data/classes.dict",
        "load_path": "./data/classes.dict",
        "in": "y",
        "out": "y_ids"
      },
      {
        "in": "x",
        "out": "x_tok",
        "id": "my_tokenizer",
        "class_name": "nltk_tokenizer",
        "tokenizer": "wordpunct_tokenize"
      },
      {
        "in": "x_tok",
        "out": "x_emb",
        "id": "my_embedder",
        "class_name": "glove",
        "load_path": "./glove.6B.100d.txt",
        "dim": 100,
        "pad_zero": True
      },
      {
        "in": "y_ids",
        "out": "y_onehot",
        "class_name": "one_hotter",
        "depth": "#classes_vocab.len",
        "single_vector": True
      },
      {
        "in": [
          "x_emb"
        ],
        "in_y": [
          "y_onehot"
        ],
        "out": [
          "y_pred_probas"
        ],
        "main": True,
        "class_name": "keras_classification_model",
        "last_layer_activation": "sigmoid",
        "save_path": "./cnn_model_v1",
        "load_path": "./cnn_model_v1",
        "embedding_size": "#my_embedder.dim",
        "n_classes": "#classes_vocab.len",
        "kernel_sizes_cnn": [
          1,
          2,
          3
        ],
        "filters_cnn": 256,
        "optimizer": "Adam",
        "learning_rate": 0.01,
        "learning_rate_decay": 0.1,
        "loss": "categorical_crossentropy",
        "coef_reg_cnn": 1e-4,
        "coef_reg_den": 1e-4,
        "dropout_rate": 0.5,
        "dense_size": 100,
        "model_name": "cnn_model"
      },
      {
        "in": "y_pred_probas",
        "out": "y_pred_ids",
        "class_name": "proba2labels",
        "top_n": 5
      },
      {
        "in": "y_pred_ids",
        "out": "y_pred_labels",
        "ref": "classes_vocab"
      }
    ],
    "out": [
      "y_pred_labels"
    ]
  },
  "train": {
    "epochs": 10,
    "batch_size": 64,
    # "metrics": [
    #   "sets_accuracy",
    #   "f1_macro",
    #   {
    #     "name": "roc_auc",
    #     "inputs": ["y_onehot", "y_pred_probas"]
    #   }
    # ],
    "validation_patience": 5,
    "val_every_n_epochs": 1,
    "log_every_n_epochs": 1,
    "show_examples": True,
    "validate_best": True,
    "test_best": False
  }
}


In [12]:
m = train_model(cnn_config)

2022-06-21 16:27:01.595 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/data/valid.csv file
2022-06-21 16:27:01.598 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/data/test.csv file
2022-06-21 16:27:01.603 INFO in 'deeppavlov.dataset_iterators.basic_classification_iterator'['basic_classification_iterator'] at line 74: Splitting field <<train>> to new fields <<['train', 'valid']>>
2022-06-21 16:27:01.608 WARNING in 'deeppavlov.core.commands.train'['train'] at line 108: "validate_best" and "test_best" parameters are deprecated. Please, use "evaluation_targets" list instead
2022-06-21 16:27:01.612 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/data/classes.dict]
2022-06-21 16:27:01.636 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [sav

{"valid": {"eval_examples_count": 100, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01", "examples": [{"x": "Вне зоны доступа: аномальные территории связи", "y_predicted": ["лазерная резка", "нииси ран", "кожа", "регулярка", "робот"], "y_true": ["зоны слабого приёма", "низкий уровень сигнала"]}, {"x": "Повседневная Теория решения изобретательских задач", "y_predicted": ["natural language processing", "basic", "автомобильная навигация", "gamification", "привлечение пользователей"], "y_true": ["ТРИЗ", "занимательные задачки"]}, {"x": "Google DeepMind и Blizzard превратят StarCraft 2 в среду для изучения ИИ", "y_predicted": ["регулярка", "платёжные системы", "ламповая электроника", "swift", "маркетинговый анализ"], "y_true": ["blizzard", "deepmind", "skynet", "star craft 2", "игры", "alpha go", "искусственный интеллект"]}, {"x": "Лень двигатель прогресса. Генератор задач. Часть 2", "y_predicted": ["регулярка", "кожа", "ONTAP", "infineon", "C#"], "y_true": ["преподавание", "алгоритмы"

2022-06-21 16:27:29.804 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the accuracy of 0.0


{"train": {"eval_examples_count": 64, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:04", "examples": [{"x": "В эксперименте с технологией 5G миллиметровые волны преодолели более 10 километров", "y_predicted": ["infineon", "jumpstart", "астрономию в массы", "vas experts", "letsencrypt"], "y_true": ["сотовая связь", "5g", "миллиметровые волны"]}, {"x": "Процедурная растительность на OpenGL и GLSL", "y_predicted": ["jumpstart", "ардуино головного мозга", "infineon", "скоростная сборка", "plugin"], "y_true": ["opengl", "glsl", "процедурная генерация", "трава"]}, {"x": "Минкомсвязь предлагает обязать юрлиц передавать сведения об IP-адресах в новую госсистему", "y_predicted": ["вежливость", "ит", "ардуино головного мозга", "letsencrypt", "Дэниель Стенберг"], "y_true": ["новости"]}, {"x": "DRP-план в ИТ-компании и проверка его работоспособности", "y_predicted": ["вежливость", "ардуино головного мозга", "ит", "letsencrypt", "jumpstart"], "y_true": ["дата-центр", "drp--план", "хьюстон у нас

2022-06-21 16:27:30.754 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the accuracy of 0.0


...

2022-06-21 16:27:33.668 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the accuracy of 0.0
2022-06-21 16:27:33.672 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 329: Ran out of patience
2022-06-21 16:27:33.741 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/data/classes.dict]
2022-06-21 16:27:33.756 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/content/glove.6B.100d.txt`]


{"train": {"eval_examples_count": 64, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:07", "examples": [{"x": "Все о том, как строится самый высокий небоскреб Европы", "y_predicted": ["машинное обучение", "emacs", "нейронные сети", "психология", "org-mode"], "y_true": ["лахта центр", "небоскреб", "петербург", "welcome", "архитектура", "строительство", "мегастройка"]}, {"x": "Космические жилища, ч. 3: как мы будем жить на Марсе", "y_predicted": ["астрономию в массы", "c++", "извечное противостояние", "квантизация", "ночное небо"], "y_true": ["космос", "жилища", "космические жилища", "космические колонии", "Марс"]}, {"x": "Итоги контеста PwnFest", "y_predicted": ["космонавтика", "фото", "comfortway", "мозг", "launcher"], "y_true": ["security"]}, {"x": "Security Week 45: обход двухфакторной авторизации в OWA, перехват аккаунтов GMail, уязвимость в OpenSSL", "y_predicted": ["квантизация", "infineon", "астрономию в массы", "c++", "Panda Email Protection"], "y_true": ["klsw", "openssl", "o

2022-06-21 16:27:58.162 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 216: [initializing `KerasClassificationModel` from scratch as cnn_model]
2022-06-21 16:27:58.845 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 129: Model was successfully initialized!
Model summary:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 100)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, None, 256)    25856       ['input_1[0][0]']                
                                                                                                  
 con

{"valid": {"eval_examples_count": 100, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01", "examples": [{"x": "Вне зоны доступа: аномальные территории связи", "y_predicted": ["связь в космосе", "эргономика", "vmware esxi", "API", "видеонаблюдение через интернет"], "y_true": ["зоны слабого приёма", "низкий уровень сигнала"]}, {"x": "Повседневная Теория решения изобретательских задач", "y_predicted": ["natural language processing", "basic", "автомобильная навигация", "gamification", "привлечение пользователей"], "y_true": ["ТРИЗ", "занимательные задачки"]}, {"x": "Google DeepMind и Blizzard превратят StarCraft 2 в среду для изучения ИИ", "y_predicted": ["связь в космосе", "dpi", "dataline", "AlphaGo", "seo-оптимизация"], "y_true": ["blizzard", "deepmind", "skynet", "star craft 2", "игры", "alpha go", "искусственный интеллект"]}, {"x": "Лень двигатель прогресса. Генератор задач. Часть 2", "y_predicted": ["mirai", "cih", "API", "vmware esxi", "embedded"], "y_true": ["преподавание", "алг

2022-06-21 16:28:23.563 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 216: [initializing `KerasClassificationModel` from scratch as cnn_model]
2022-06-21 16:28:24.232 INFO in 'deeppavlov.models.classifiers.keras_classification_model'['keras_classification_model'] at line 129: Model was successfully initialized!
Model summary:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 100)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, None, 256)    25856       ['input_1[0][0]']                
                                                                                                  
 con

In [15]:
url = 'https://raw.githubusercontent.com/Kukarkin/Digital-Student-Assistant-Recommender/master/data/translated_preprocessed_names.csv'
df = pd.read_csv(url, index_col=0)
df.head()

,0
0,Новая концепция для музея «Микула» в Нижнем Но...
1,Развитие туристического кластера в Кроноцком з...
2,Создание мобильного приложения СППР для диагно...
3,Гуманитарная политика России
4,Подкаст про современное высшее образование


In [25]:
df = pd.read_csv("./data/train.csv")
names = list(df["title"])
names = ast.literal_eval(str(names))

In [28]:
predicted = m(names)

In [29]:
fin = pd.DataFrame(predicted)
fin2 = pd.DataFrame(names)
result = pd.concat([fin2, fin], axis=1)

In [31]:
result.head(10)

,0,0,1,2,3,4
0,Опыт использования MassTransit 3.0,ИТ-индустрия,пять почему,медный кабель,Yii 2,PayOnline
1,Геймификация форума на движке XenForo,гаджеты,vmfs,tutorial,iridium,умный автомобиль
2,Кластер высокой доступности на postgresql 9.6 ...,lenovo,vmfs,гаджеты,PayOnline,Корпоративные лаборатории Pentestit
3,Как перестать бояться и полюбить синтаксически...,vmfs,tutorial,украина,image classification,ИТ-индустрия
4,Мифы и реальность: Что нужно знать о современн...,vmfs,геллер,tutorial,image classification,Imagination Technologies
5,Как выбрать тему для email-рассылки: универсал...,sdn,tutorial,ВИЧ,lenovo,mongo
6,Как мы отказоустойчивый кластер запускали,natural language processing,basic,автомобильная навигация,gamification,привлечение пользователей
7,Deep Packet Inspection: Оборудование и применение,vmfs,image classification,гаджеты,tutorial,умный автомобиль
8,Удалённое включение скриптов Mikrotik из Telegram,гаджеты,vmfs,embedded,снг,заказная разработка
9,Как заставить отдел продаж дочитать скучное пи...,vmfs,tutorial,украина,здоровье,Imagination Technologies
